# Model Deployment using Tensorflow

## 1. Introduction
In this workbook, we will train a simple Tensorflow model and deploy that for inference. 
In this example, we use TensorFlow's [premade estimator iris data example](https://www.tensorflow.org/tutorials/estimator/premade) and add MLflow tracking.
This example trains a `tf.estimator.DNNClassifier` on the [iris dataset](https://archive.ics.uci.edu/ml/datasets/iris) and predicts on a validation set.
We then demonstrate how to load the saved model back as a generic `mlflow.pyfunc`, allowing us to make predictions.


## 2. Imports and Dependencies.
The few packages needed are loaded next. Particularly, `tensorflow`, `mlflow` will be majorly used in this tutorial. `requests` package will be used for performing query. `json` is used to post and get response from the server.

In [1]:
import os
import sys
import mlflow
import numpy as np
import mlflow.tensorflow
from mlflow import pyfunc
import tensorflow as tf
import pandas as pd
import tempfile
import shutil

# Setting a tracking uri to log the mlflow logs in a particular location tracked by 
from mlflow.tracking import MlflowClient
tracking_uri = os.environ.get("TRACKING_URL")
client = MlflowClient(tracking_uri=tracking_uri)
mlflow.set_tracking_uri(tracking_uri)

# Setting a tracking project experiment name to keep the experiments organized
experiments = client.list_experiments()
experiment_names = []
for exp in experiments:
    experiment_names.append(exp.name)
experiment_name = "tf_development"
if experiment_name not in experiment_names:
    mlflow.create_experiment(experiment_name)
mlflow.set_experiment(experiment_name)

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

2021-11-13 23:40:49.979783: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-13 23:40:49.979819: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# 3. Some utility functions

In [2]:
## Utility function to add libraries to conda environment
def add_libraries_to_conda_env(_conda_env,libraries=[],conda_dependencies=[]):
    dependencies = _conda_env["dependencies"]
    dependencies = dependencies + conda_dependencies
    pip_index = None
    for _index,_element in enumerate(dependencies):
        if type(_element) == dict:
            if "pip" in _element.keys():
                pip_index = _index
                break
    dependencies[pip_index]["pip"] =  dependencies[pip_index]["pip"] + libraries
    _conda_env["dependencies"] = dependencies
    return _conda_env

TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ["SepalLength", "SepalWidth", "PetalLength", "PetalWidth", "Species"]
SPECIES = ["Setosa", "Versicolor", "Virginica"]


def load_data(y_name="Species"):
    """Returns the iris dataset as (train_x, train_y), (test_x, test_y)."""
    train_path = tf.keras.utils.get_file(TRAIN_URL.split("/")[-1], TRAIN_URL)
    test_path = tf.keras.utils.get_file(TEST_URL.split("/")[-1], TEST_URL)

    train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
    train_x, train_y = train, train.pop(y_name)

    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
    test_x, test_y = test, test.pop(y_name)

    return (train_x, train_y), (test_x, test_y)


def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset


def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features = dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset


In [3]:
mlflow.tensorflow.autolog()
batch_size = 100
train_steps = 1000
runid = mlflow.start_run()

In [4]:
# Fetch the data
(train_x, train_y), (test_x, test_y) = load_data()

# Feature columns describe how to use the input.
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [5]:
# Two hidden layers of 10 nodes each.
hidden_units = [10, 10]

# Build 2 hidden layer DNN with 10, 10 units respectively.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=hidden_units,
    # The model must choose between 3 classes.
    n_classes=3,
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpk_6blyoi', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


2021-11-13 23:40:53.870110: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-13 23:40:53.870158: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-13 23:40:53.870196: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bneievrk-76946f66f-dtbkv): /proc/driver/nvidia/version does not exist
2021-11-13 23:40:53.871087: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# Train the Model.
estimator = classifier.train(
    input_fn=lambda: train_input_fn(train_x, train_y, batch_size),
    steps=train_steps,
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpk_6blyoi/model.ckpt-1000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 1000...
INFO:tensorflow:Saving checkpoints for 1000 into /tmp/tmpk_6blyoi/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 1000...
INFO:tensorflow:loss = 0.82140183, step = 1000
INFO:tensorflow:global_step/sec: 55.685
INFO:tensorflow:loss = 0.7917734, step = 1100 (2.486 sec)
INFO:tensorflow:global_step/sec: 41.0757
INFO:tensorflow:loss = 0.7669014, step = 1200 (2.338 sec)
INFO:tensorflow:global_step/sec: 42.6867
INFO:tensorflow:loss = 0.7893652, step = 1300 (2.157 sec)
INFO:tensorflow:global_step/sec: 46.3216
I

In [8]:
# Evaluate the model.
eval_result = classifier.evaluate(
    input_fn=lambda: eval_input_fn(test_x, test_y, batch_size)
)

print("\nTest set accuracy: {accuracy:0.3f}\n".format(**eval_result))

# Generate predictions from the model
expected = ["Setosa", "Versicolor", "Virginica"]
predict_x = {
    "SepalLength": [5.1, 5.9, 6.9],
    "SepalWidth": [3.3, 3.0, 3.1],
    "PetalLength": [1.7, 4.2, 5.4],
    "PetalWidth": [0.5, 1.5, 2.1],
}

predictions = classifier.predict(
    input_fn=lambda: eval_input_fn(predict_x, labels=None, batch_size=batch_size)
)

old_predictions = []
template = '\nPrediction is "{}" ({:.1f}%), expected "{}"'

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-11-13T23:42:07
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpk_6blyoi/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.19000s
INFO:tensorflow:Finished evaluation at 2021-11-13-23:42:07
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.73333335, average_loss = 0.7398704, global_step = 2000, loss = 0.7398704
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2000: /tmp/tmpk_6blyoi/model.ckpt-2000

Test set accuracy: 0.733



In [9]:
for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict["class_ids"][0]
    probability = pred_dict["probabilities"][class_id]

    print(template.format(SPECIES[class_id], 100 * probability, expec))

    old_predictions.append(SPECIES[class_id])

# Creating output tf.Variables to specify the output of the saved model.
feat_specifications = {
    "SepalLength": tf.Variable([], dtype=tf.float64, name="SepalLength"),
    "SepalWidth": tf.Variable([], dtype=tf.float64, name="SepalWidth"),
    "PetalLength": tf.Variable([], dtype=tf.float64, name="PetalLength"),
    "PetalWidth": tf.Variable([], dtype=tf.float64, name="PetalWidth"),
}

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpk_6blyoi/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "Setosa" (33.7%), expected "Setosa"

Prediction is "Virginica" (50.6%), expected "Versicolor"

Prediction is "Virginica" (66.1%), expected "Virginica"


In [10]:
receiver_fn = tf.estimator.export.build_raw_serving_input_receiver_fn(feat_specifications)
temp = tempfile.mkdtemp()
try:
    # The model is automatically logged when export_saved_model() is called.
    saved_estimator_path = classifier.export_saved_model(temp, receiver_fn).decode("utf-8")
    mlflow.end_run()
    # Since the model was automatically logged as an artifact (more specifically
    # a MLflow Model), we don't need to use saved_estimator_path to load back the model.
    # MLflow takes care of it!
    pyfunc_model = pyfunc.load_model(mlflow.get_artifact_uri("tf_development"))

    predict_data = [[5.1, 3.3, 1.7, 0.5], [5.9, 3.0, 4.2, 1.5], [6.9, 3.1, 5.4, 2.1]]
    df = pd.DataFrame(
        data=predict_data,
        columns=["SepalLength", "SepalWidth", "PetalLength", "PetalWidth"],
    )

    # Predicting on the loaded Python Function and a DataFrame containing the
    # original data we predicted on.
    predict_df = pyfunc_model.predict(df)

    # Checking the PyFunc's predictions are the same as the original model's predictions.
    template = '\nOriginal prediction is "{}", reloaded prediction is "{}"'
    for expec, pred in zip(old_predictions, predict_df["classes"]):
        class_id = predict_df["class_ids"][
            predict_df.loc[predict_df["classes"] == pred].index[0]
        ]
        reloaded_label = SPECIES[class_id]
        print(template.format(expec, reloaded_label))
finally:
    shutil.rmtree(temp)

2021/11/13 23:42:10 INFO mlflow.tensorflow: Logging TensorFlow Estimator as MLflow Model to run with ID '8a7036efeb454117bfbed4eec5a529db'


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Signatures EXCLUDED from export because they cannot be be served via TensorFlow Serving APIs:
INFO:tensorflow:'serving_default' : Classification input must be a single string Tensor; got {'SepalLength': <tf.Tensor 'Placeholder:0' shape=(None,) dtype=float64>, 'SepalWidth': <tf.Tensor 'Placeholder_1:0' shape=(None,) dtype=float64>, 'PetalLength': <tf.Tensor 'Placeholder_2:0' shape=(None

2021/11/13 23:42:11 INFO mlflow.tensorflow: Validating the specified TensorFlow model by attempting to load it in a new TensorFlow graph...
2021-11-13 23:42:11.302002: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021/11/13 23:42:11 INFO mlflow.tensorflow: Validation succeeded!


ResourceNotFoundError: The specified blob does not exist.
RequestId:6155ab77-301e-0008-31e8-d84fb0000000
Time:2021-11-13T23:42:16.4419763Z
ErrorCode:BlobNotFound

## 4. Deploying the model
The above code logs a model in the experiments tab. For more info please refer [here](https://rocketml.gitbook.io/rocketml-user-guide/experiments). After deploying the model, we can obtain the model url for performing query as shown below.

## 5. Query from the server

There are two methods to perform query... The first is using `requests` library and the other using `curl` shell command.

In [ ]:
import requests
import json

url = "http://127.0.0.1:5011/invocations"
headers = {"Content-Type":"text/csv"}

# First case, run inference on single data point
np_array = np.random.rand(1,6).tolist()
json_data = json.dumps(np_array)
response = requests.post(url,data=json_data,headers=headers)
if response.status_code == 200:
    output = np.array(json.loads(response.json())).astype(np.float32)
    print(output)
else:
    print(response.status_code)
    print("REST API deployment is in progress -- please try again in a few minutes!")

# Second case, run inference on multiple data points
np_array = np.random.rand(20,6).tolist()
json_data = json.dumps(np_array)
response = requests.post(url,data=json_data,headers=headers)
if response.status_code == 200:
    output = np.array(json.loads(response.json())).astype(np.float32)
    print(output)
else:
    print(response.status_code)
    print("REST API deployment is in progress -- please try again in a few minutes!")

In [ ]:
!curl http://127.0.0.1:5011/invocations -H 'Content-Type:text/csv' -d '[[0.6499166977064089, 0.17579454262114602, 0.2688911143313131, 0.7146591854799202, 0.6497433572112488, 0.7723469203958951]]'